<a href="https://colab.research.google.com/github/cardavid11/tensorflow_certification/blob/main/graphs_and_tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to graphs and tf.function

For an interview, it's important to have a clear and concise understanding of TensorFlow graphs and `tf.function`. Here's a summarized explanation:

1. **TensorFlow Graphs**: In TensorFlow, a graph represents a series of computations. Each computation (like mathematical operations) is a node, and the data (tensors) flowing between them are edges. Graphs enable efficient execution by optimizing calculations, facilitating parallel processing, and allowing operations to be performed independently of the Python runtime.

2. **`tf.function`**: This is a TensorFlow decorator that converts a Python function into a TensorFlow graph. It allows for normal Python code, including control flow statements, to be automatically transformed into TensorFlow's graph operations. This conversion optimizes the function for performance and is crucial for deploying TensorFlow models in non-Python environments.

3. **AutoGraph**: Part of `tf.function`, AutoGraph converts Python-specific constructs (like loops and conditional statements) into TensorFlow graph-compatible code. This ensures that Python functions work seamlessly in a graph environment.

Understanding these concepts is key for TensorFlow models, as they impact performance optimization and model deployment.



In [ ]:
# What are graphs? -> Graphs are data structures that contain a set of tf.Operation
# objects, which represent units of computation; and tf.Tensor objects, which represent
# the units of data that flow between operations. They are defined in a tf.Graph context.
# Since these graphs are data structures, they can be saved, run, and restored all
# without the original Python code.

In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

In [7]:
# A Function is a Python callable that builds TensorFlow graphs from the Python
# function. You use a Function in the same way as its Python equivalent.
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()

# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

In [8]:
# tf.function applies to a function and all other functions it calls:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

In [ ]:
# Converting Python functions to graphs

# Any function you write with TensorFlow will contain a mixture of built-in TF operations
# and Python logic, such as if-then clauses, loops, break, return, continue, and more.
# While TensorFlow operations are easily captured by a tf.Graph, Python-specific logic
# needs to undergo an extra step in order to become part of the graph. tf.function
# uses a library called AutoGraph (tf.autograph) to convert Python code into
# graph-generating code.

def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())